In [ ]:
!pip install transformers torch nltk pymupdf --quiet

In [ ]:
import fitz
import nltk
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from google.colab import files

In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

18072025-RIL-Media-Analyst-Call-Transcript-Q1FY2026.pdf
18072025-RIL-Media-Analyst-Call-Transcript-Q1FY2026.pdf(application/pdf) - 441321 bytes, last modified: 08/09/2025 - 100% done
Saving 18072025-RIL-Media-Analyst-Call-Transcript-Q1FY2026.pdf to 18072025-RIL-Media-Analyst-Call-Transcript-Q1FY2026.pdf

In [ ]:
doc = fitz.open(pdf_path)
text = ""
for page in doc:
    text += page.get_text()

In [ ]:
nltk.download("punkt", quiet=True)
nltk.download("punkt_tab", quiet=True)
sents = nltk.sent_tokenize(text)
sents = [s.strip() for s in sents if 20 <= len(s) <= 1000]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
model.eval()

In [ ]:
p_pos, p_neg, p_neu = [], [], []
batch = 16
for i in range(0, len(sents), batch):
    chunk = sents[i:i+batch]
    with torch.no_grad():
        enc = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True, max_length=256)
        logits = model(**enc).logits
        probs = torch.softmax(logits, dim=-1).cpu().numpy()
    p_pos.extend(probs[:,0].tolist())
    p_neg.extend(probs[:,1].tolist())
    p_neu.extend(probs[:,2].tolist())

In [ ]:
p_pos = float(np.mean(p_pos))
p_neg = float(np.mean(p_neg))
p_neu = float(np.mean(p_neu))
net = p_pos - p_neg
print(f"net={net:.3f} pos={p_pos:.3f} neg={p_neg:.3f} neu={p_neu:.3f}")

net=0.232 pos=0.319 neg=0.087 neu=0.595